In [ ]:
# Hack to make the module importable
import sys
sys.path.append(r'./../')

In [ ]:
%load_ext autoreload
%autoreload 2
from py2neo import Graph
import requests
import pyodata

from rel2graph.relational_modules.odata import ODataListIterator
from rel2graph import Converter, register_attribute_postprocessor, Attribute
import rel2graph.common_modules
from rel2graph.utils import load_file

In [ ]:
# Configure Logging
import logging

#logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger("rel2graph")
logger.setLevel(logging.DEBUG)
log_formatter = logging.Formatter("%(asctime)s [%(threadName)s]::[%(levelname)s] : %(message)s")
console_handler = logging.StreamHandler()
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)

In [ ]:
graph = Graph(scheme="bolt", host="localhost", port=7687,  auth=('neo4j', 'password'))

graph.delete_all()  

In [ ]:
###### CONFIGURATIONS #########
SERVICE_URL = 'https://ws.parlament.ch/odata.svc/'
HTTP_LIB = requests.Session()
client = pyodata.Client(SERVICE_URL, HTTP_LIB)

# Initialize node and relation dumps
nodes = []
relations = []

target = client.entity_sets.Person.get_entity(ID=4049,Language="DE").execute()

In [ ]:
@register_attribute_postprocessor
def APPEND(attribute, string):
    new_attr = Attribute(attribute.key, attribute.value + string)
    return new_attr

@register_attribute_postprocessor
def ASSERT_EQ(attribute: Attribute, value: str) -> Attribute:
    assert attribute.value == value, f"Attribute {attribute.key} '{attribute.value}' != '{value}'"
    return attribute

In [ ]:
iterator = ODataListIterator([target])

In [ ]:
schema_config = "./odata_demo_config.yaml"
converter = Converter(load_file(schema_config), iterator, graph, num_workers=1)

In [ ]:
converter()